In [ ]:
import pysam
import pandas as pd
import numpy as np
from collections import defaultdict
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

In [ ]:
# Load necessary R packages
pandas2ri.activate()
deseq = importr("DESeq2")

# Paths to BAM file and gene annotation file (GTF/GFF format)
bam_file = 'path_to_your.bam'
annotation_file = 'path_to_your_annotation.gtf'


In [ ]:
# Load gene annotation into a dictionary
gene_dict = defaultdict(list)
with open(annotation_file, 'r') as f:
    for line in f:
        if line.startswith('#'):
            continue
        parts = line.strip().split('\t')
        if parts[2] == 'gene':
            gene_id = parts[8].split(';')[0].split(' ')[1].replace('"', '')
            gene_name = parts[8].split(';')[1].split(' ')[2].replace('"', '')
            gene_dict[gene_id] = gene_name


In [ ]:
# Initialize a dictionary to store read counts per gene
gene_read_counts = defaultdict(int)

# Open the BAM file using pysam
bam = pysam.AlignmentFile(bam_file, 'rb')

# Count reads per gene
for read in bam:
    if read.is_unmapped:
        continue
    gene_id = read.get_reference_name()
    gene_read_counts[gene_id] += 1


In [ ]:
# Convert the read counts dictionary into a pandas DataFrame
counts_df = pd.DataFrame(gene_read_counts.items(), columns=['GeneID', 'ReadCounts'])

# Load annotation information into the DataFrame
counts_df['GeneName'] = counts_df['GeneID'].apply(lambda x: gene_dict.get(x, 'Unknown'))

In [ ]:
# Normalize counts to TPM
total_reads = counts_df['ReadCounts'].sum()
counts_df['TPM'] = (counts_df['ReadCounts'] / total_reads) * 1e6

# Create a DESeq2-like count matrix with samples as columns and genes as rows
count_matrix = counts_df.pivot(index='GeneName', columns='Sample', values='ReadCounts')

# Perform differential expression analysis using DESeq2
dds = deseq.DESeqDataSetFromMatrix(countData=count_matrix, colData=col_data, design=~condition)
dds = deseq.DESeq(dds)
results = deseq.results(dds)
results_df = pandas2ri.ri2py(results)

# Print the top differentially expressed genes
print(results_df.head())
